# Automated Guitar Amp Modelling Training on Google Colab

This notebook sets up and runs the neural network training for guitar amplifier/distortion pedal modelling using PyTorch on Google Colab GPUs.

## Setup Instructions:
1. **Enable GPU**: Runtime → Change runtime type → Hardware accelerator → **GPU** (T4 or better)
2. **Upload Dataset**: Upload your training data as a ZIP file containing the Data folder structure
3. **Run All Cells**: Run cells in order from top to bottom
4. **Monitor Training**: Use the GPU monitoring cell to track GPU usage during training

## Dataset Structure:
Your ZIP file should contain:
```
Data/
  ├── train/
  │   ├── dls2-input.wav
  │   └── dls2-target.wav
  ├── val/
  │   ├── dls2-input.wav
  │   └── dls2-target.wav
  └── test/
      ├── dls2-input.wav
      └── dls2-target.wav
```

In [ ]:
# Check GPU availability and setup
import torch
import os

print("=" * 60)
print("GPU Setup Check")
print("=" * 60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"Current GPU Memory Allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"Current GPU Memory Cached: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")
    
    # Set device
    device = torch.device('cuda')
    print(f"\n✅ Using device: {device}")
    
    # Clear any existing GPU cache
    torch.cuda.empty_cache()
    print("✅ GPU cache cleared")
else:
    print("⚠️  WARNING: No GPU detected!")
    print("   Please enable GPU: Runtime → Change runtime type → Hardware accelerator → GPU")
    print("   Training will be VERY slow on CPU!")
    device = torch.device('cpu')

print("=" * 60)

In [ ]:
# Install required dependencies for GPU training
print("Installing PyTorch with CUDA support...")
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 -q

print("\nInstalling audio processing libraries...")
%pip install numpy scipy matplotlib pyyaml tqdm librosa tensorboard -q

print("\nInstalling additional utilities...")
%pip install psutil -q

print("\n✅ All dependencies installed!")
print("\nVerifying CUDA installation...")
import torch
if torch.cuda.is_available():
    print(f"✅ CUDA {torch.version.cuda} is working!")
else:
    print("⚠️  CUDA not available - check GPU runtime settings")

In [ ]:
# Clone or upload the repository
import os
import shutil

print("=" * 60)
print("Repository Setup")
print("=" * 60)

# Check if we're already in the repo directory
if os.path.exists('CoreAudioML') and os.path.exists('dist_model_recnet.py'):
    print("✅ Repository files already present in current directory")
    repo_path = os.getcwd()
    print(f"   Working directory: {repo_path}")
else:
    # Try to clone the repository - use the correct repository name
    repo_name = 'MonsterLSTM_Capture'
    
    # Clean up any old clones
    if os.path.exists(repo_name):
        print(f"Removing old {repo_name} directory...")
        shutil.rmtree(repo_name, ignore_errors=True)
    
    # Also check for old Automated-GuitarAmpModelling-3 directory
    if os.path.exists('Automated-GuitarAmpModelling-3'):
        print("Removing old Automated-GuitarAmpModelling-3 directory...")
        shutil.rmtree('Automated-GuitarAmpModelling-3', ignore_errors=True)
    
    # Clone the repository from the correct GitHub URL
    print(f"Cloning repository from GitHub...")
    !git clone https://github.com/diogoguedes666/MonsterLSTM_Capture.git
    
    if os.path.exists(repo_name):
        # Change to the repository directory
        # Note: os.chdir() persists across cells in Colab
        os.chdir(repo_name)
        repo_path = os.getcwd()
        print(f"✅ Repository cloned successfully")
        print(f"   Working directory: {repo_path}")
    else:
        print("⚠️  Repository clone failed!")
        print("   Please check your internet connection or upload files manually")
        repo_path = os.getcwd()

# Verify we're in the right directory
print(f"\n📋 Verification:")
print(f"   Current directory: {os.getcwd()}")
print(f"   CoreAudioML exists: {os.path.exists('CoreAudioML')}")
print(f"   dist_model_recnet.py exists: {os.path.exists('dist_model_recnet.py')}")

if os.path.exists('CoreAudioML'):
    print(f"\n✅ CoreAudioML module found!")
    core_files = os.listdir('CoreAudioML')
    print(f"   Module files: {', '.join(core_files[:5])}")
    if len(core_files) > 5:
        print(f"   ... and {len(core_files) - 5} more files")
else:
    print("\n⚠️  CoreAudioML not found!")
    print(f"   Current directory contents:")
    for item in os.listdir('.')[:10]:
        print(f"      - {item}")

print("=" * 60)

In [ ]:
# Create necessary directories
import os

dirs_to_create = [
    'Data/train',
    'Data/val', 
    'Data/test',
    'Results',
    'Configs'
]

for dir_path in dirs_to_create:
    os.makedirs(dir_path, exist_ok=True)
    print(f"✅ Created: {dir_path}")

print("\n✅ All directories created!")

In [ ]:
# Option 1: Generate test signals if you don't have real training data
# This creates synthetic audio data for testing the training pipeline

import numpy as np
import scipy.io.wavfile as wavfile
import os

def generate_test_audio(filename, duration=10, sample_rate=44100, freq=440):
    """Generate a simple test audio file"""
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    # Create a mix of sine waves for more interesting content
    audio = 0.3 * np.sin(freq * 2 * np.pi * t)  # 440 Hz sine wave
    audio += 0.2 * np.sin(2 * freq * 2 * np.pi * t)  # 880 Hz harmonic
    audio += 0.1 * np.sin(3 * freq * 2 * np.pi * t)  # 1320 Hz harmonic

    # Normalize to prevent clipping
    audio = audio / np.max(np.abs(audio)) * 0.8

    # Convert to 16-bit PCM
    audio_int16 = (audio * 32767).astype(np.int16)

    wavfile.write(filename, sample_rate, audio_int16)
    print(f"Generated test audio: {filename}")

# Generate test data
generate_test_audio("Data/train/dls2-input.wav", duration=30)
generate_test_audio("Data/train/dls2-target.wav", duration=30)  # This would be your "amp output"

generate_test_audio("Data/val/dls2-input.wav", duration=10)
generate_test_audio("Data/val/dls2-target.wav", duration=10)

generate_test_audio("Data/test/dls2-input.wav", duration=5)
generate_test_audio("Data/test/dls2-target.wav", duration=5)

print("Test data generation complete!")
print("Note: For real amp modelling, replace these with actual guitar -> amp recordings")

In [ ]:
# Upload your training dataset as a ZIP file
from google.colab import files
import os
import zipfile
import shutil

print("=" * 60)
print("Dataset Upload")
print("=" * 60)
print("\n📦 Please upload your ZIP file containing the Data folder")
print("\nYour ZIP file should have this structure:")
print("  Data.zip")
print("    └── Data/")
print("        ├── train/")
print("        │   ├── dls2-input.wav")
print("        │   └── dls2-target.wav")
print("        ├── val/")
print("        │   ├── dls2-input.wav")
print("        │   └── dls2-target.wav")
print("        └── test/")
print("            ├── dls2-input.wav")
print("            └── dls2-target.wav")
print("\n" + "=" * 60)

# Upload ZIP file
print("\n📤 Click 'Choose Files' to upload your dataset ZIP file...")
uploaded = files.upload()

# Find the ZIP file
zip_filename = None
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        zip_filename = filename
        print(f"\n✅ Found ZIP file: {zip_filename}")
        break

if not zip_filename:
    print("❌ No ZIP file found in upload!")
    print("   Please make sure you uploaded a .zip file")
else:
    # Extract ZIP file
    print(f"\n📂 Extracting {zip_filename}...")
    try:
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print("✅ Extraction complete!")
    except Exception as e:
        print(f"❌ Error extracting ZIP: {e}")
        raise
    
    # Check if Data folder exists (handle different ZIP structures)
    data_path = None
    if os.path.exists('Data'):
        data_path = 'Data'
    elif os.path.exists(os.path.join(zip_filename.replace('.zip', ''), 'Data')):
        # ZIP might have extracted to a subfolder
        data_path = os.path.join(zip_filename.replace('.zip', ''), 'Data')
        # Move Data folder to root
        shutil.move(data_path, 'Data')
        data_path = 'Data'
    
    if data_path and os.path.exists(data_path):
        print(f"\n✅ Data folder found at: {data_path}")
        
        # Verify required files
        required_files = [
            'Data/train/dls2-input.wav',
            'Data/train/dls2-target.wav',
            'Data/val/dls2-input.wav',
            'Data/val/dls2-target.wav',
            'Data/test/dls2-input.wav',
            'Data/test/dls2-target.wav'
        ]
        
        missing = []
        found = []
        for filepath in required_files:
            if os.path.exists(filepath):
                found.append(filepath)
            else:
                missing.append(filepath)
        
        print(f"\n📊 File verification:")
        print(f"   ✅ Found: {len(found)}/{len(required_files)} files")
        
        if missing:
            print(f"\n⚠️  Missing files:")
            for f in missing:
                print(f"   - {f}")
            print("\n⚠️  Training may fail without all required files!")
        else:
            print("\n✅ All required files found!")
        
        # Show data structure
        print("\n📁 Data structure:")
        for split in ['train', 'val', 'test']:
            split_path = f'Data/{split}'
            if os.path.exists(split_path):
                files_in_split = os.listdir(split_path)
                print(f"   {split_path}/: {len(files_in_split)} file(s)")
                for f in files_in_split[:3]:  # Show first 3 files
                    print(f"      - {f}")
                if len(files_in_split) > 3:
                    print(f"      ... and {len(files_in_split) - 3} more")
    else:
        print("⚠️  Data folder not found in ZIP!")
        print(f"   Extracted contents: {os.listdir('.')[:10]}")
        print("\n   Please check your ZIP file structure.")
    
    # Clean up zip file
    if os.path.exists(zip_filename):
        os.remove(zip_filename)
        print(f"\n🗑️  Cleaned up: {zip_filename}")

print("\n" + "=" * 60)
print("✅ Dataset upload complete!")
print("=" * 60)

In [ ]:
# Create RNN3.json config file optimized for GPU training
import json

config = {
    "model": "SimpleRNN",
    "input_size": 1,
    "output_size": 1,
    "num_blocks": 2,
    "hidden_size": 96,  # Optimized for GPU (can handle larger models)
    "unit_type": "LSTM",
    "skip_con": True,
    "segment_length": 22050,
    "batch_size": 512,  # GPU-friendly batch size (adjust if OOM)
    "epochs": 100,
    "learn_rate": 0.005,
    "validation_f": 5,
    "validation_p": 20,
    "loss_fcns": {
        "ESR": 0.75,
        "DC": 0.10,
        "HFHinge": 0.15
    },
    "pre_filt": "None",
    "cuda": 1,  # Enable CUDA/GPU
    "weight_decay": 0.000001,
    "gradient_clip": 1.0,
    "hf_hinge_fmin": 10000,
    "hf_hinge_strength": 0.5
}

# Save config file
config_path = 'Configs/RNN3.json'
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print(f"✅ Config file created: {config_path}")
print(f"\n📋 Configuration:")
print(f"   Model: {config['model']}")
print(f"   Hidden Size: {config['hidden_size']}")
print(f"   Batch Size: {config['batch_size']}")
print(f"   Epochs: {config['epochs']}")
print(f"   Learning Rate: {config['learn_rate']}")
print(f"   GPU Enabled: {config['cuda'] == 1}")
print(f"\n💡 Tip: If you get CUDA out of memory errors, reduce batch_size to 256 or 128")

In [ ]:
# Verify data files exist
!ls -la Data/train/
!ls -la Data/val/
!ls -la Data/test/

In [ ]:
# Verify imports and setup
import sys
import os
import torch

print("=" * 60)
print("Module Import Test")
print("=" * 60)

# Ensure we're in the correct directory
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

# Check multiple possible locations for the repository
possible_paths = [
    current_dir,  # Current directory
    'MonsterLSTM_Capture',  # If we're in /content
    'Automated-GuitarAmpModelling-3',  # Old name (if exists)
    os.path.join('..', 'MonsterLSTM_Capture'),  # Parent directory
]

repo_path = None
for path in possible_paths:
    full_path = os.path.abspath(path) if not os.path.isabs(path) else path
    if os.path.exists(os.path.join(full_path, 'CoreAudioML')) and os.path.exists(os.path.join(full_path, 'dist_model_recnet.py')):
        repo_path = full_path
        print(f"✅ Found repository at: {repo_path}")
        if full_path != current_dir:
            os.chdir(full_path)
            print(f"   Changed to: {os.getcwd()}")
        break

if repo_path is None:
    print("⚠️  Repository not found in expected locations!")
    print(f"   Current directory: {current_dir}")
    print(f"   Contents: {os.listdir('.')[:10]}")
    # Try to find it by walking
    for root, dirs, files in os.walk('.'):
        if 'CoreAudioML' in dirs and 'dist_model_recnet.py' in files:
            repo_path = root
            os.chdir(root)
            print(f"✅ Found repository at: {os.path.abspath(root)}")
            break

# Add current directory to Python path
current_dir = os.getcwd()
sys.path.insert(0, current_dir)
print(f"\n✅ Added to Python path: {current_dir}")

# Verify required files exist
required_files = [
    'CoreAudioML',
    'dist_model_recnet.py',
    'Configs/RNN3.json',
    'Data/train',
    'Data/val',
    'Data/test'
]

print("\n📋 Checking required files:")
for item in required_files:
    exists = os.path.exists(item)
    status = "✅" if exists else "❌"
    print(f"   {status} {item}")

# Test imports
print("\n🔍 Testing module imports...")
try:
    import CoreAudioML.miscfuncs as miscfuncs
    print("   ✅ CoreAudioML.miscfuncs")
    
    import CoreAudioML.training as training
    print("   ✅ CoreAudioML.training")
    
    import CoreAudioML.dataset as dataset
    print("   ✅ CoreAudioML.dataset")
    
    import CoreAudioML.networks as networks
    print("   ✅ CoreAudioML.networks")
    
    print("\n✅ All CoreAudioML modules imported successfully!")
    
    # Verify GPU availability
    if torch.cuda.is_available():
        print(f"\n✅ GPU ready: {torch.cuda.get_device_name(0)}")
        print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    else:
        print("\n⚠️  GPU not available - training will be slow!")
        
except Exception as e:
    print(f"\n❌ Import error: {e}")
    print(f"   Current directory: {os.getcwd()}")
    if os.path.exists('CoreAudioML'):
        print(f"   CoreAudioML contents: {os.listdir('CoreAudioML')}")
    else:
        print("   CoreAudioML directory not found!")
        print(f"   Current directory contents:")
        for item in os.listdir('.')[:15]:
            item_path = os.path.join('.', item)
            item_type = "DIR" if os.path.isdir(item_path) else "FILE"
            print(f"      [{item_type}] {item}")
    import traceback
    traceback.print_exc()
    raise

print("=" * 60)

In [ ]:
# Start GPU Training!
# This cell runs the actual training on GPU

import torch
import gc
import os

print("=" * 60)
print("Starting GPU Training")
print("=" * 60)

# Clear GPU cache before training
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"✅ GPU cache cleared")
    print(f"   Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️  No GPU detected - training will be slow on CPU!")

# Set environment variable to ensure GPU is used
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

print("\n🚀 Starting training...")
print("   This may take a while. Monitor progress below.")
print("   You can check GPU usage in the monitoring cell (Cell 13)")
print("\n" + "=" * 60 + "\n")

# Run training with GPU support
# The script will automatically detect and use GPU if available
!python dist_model_recnet.py --load_config RNN3 --epochs 100 --device dls2 --cuda 1

# Clean up after training
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"\n✅ Training complete! GPU memory cleared.")

In [ ]:
# Optional: Continue training or run with custom parameters
# Uncomment and modify as needed

# Example 1: Run more epochs
# !python dist_model_recnet.py --load_config RNN3 --epochs 200 --device dls2 --cuda 1

# Example 2: Adjust batch size (if you get CUDA OOM errors, reduce this)
# !python dist_model_recnet.py --load_config RNN3 --epochs 100 --batch_size 256 --device dls2 --cuda 1

# Example 3: Train with larger model
# !python dist_model_recnet.py --load_config RNN3 --epochs 100 --hidden_size 128 --device dls2 --cuda 1

print("💡 Tips:")
print("   - If you get 'CUDA out of memory', reduce batch_size to 256 or 128")
print("   - Larger hidden_size (128, 256) gives better quality but uses more GPU memory")
print("   - Monitor GPU usage in Cell 13 while training")
print("   - Training checkpoints are saved every 10 epochs in Results/")

In [ ]:
# Check training results
!ls -la Results/

# If training completed, you should see directories like 'dls2-RNN3' with model files
# You can download the results:
# from google.colab import files
# !zip -r results.zip Results/
# files.download('results.zip')

In [ ]:
# Monitor GPU usage during training
# Run this cell in a separate tab while training is running to monitor GPU utilization

import time
import subprocess

print("=" * 60)
print("GPU Monitoring")
print("=" * 60)
print("\nPress Ctrl+C to stop monitoring\n")

try:
    # Monitor GPU every 5 seconds
    for i in range(12):  # Monitor for 1 minute (12 * 5 seconds)
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=timestamp,name,utilization.gpu,utilization.memory,memory.used,memory.free,temperature.gpu', 
             '--format=csv,noheader,nounits'],
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            print(f"\n[{time.strftime('%H:%M:%S')}] GPU Status:")
            print(result.stdout)
        else:
            print("⚠️  Could not query GPU. Make sure GPU runtime is enabled.")
            break
        
        time.sleep(5)
        
except KeyboardInterrupt:
    print("\n\n✅ Monitoring stopped")

print("\n" + "=" * 60)

## Tips for Better Training:

1. **GPU Memory**: If you get CUDA out of memory errors, reduce `batch_size` in the config
2. **Training Time**: Start with fewer epochs (50-100) to test, then increase
3. **Model Size**: Larger `hidden_size` (64, 128) gives better quality but slower training
4. **Data Quality**: Real guitar recordings give much better results than synthetic data
5. **Monitoring**: Use TensorBoard to monitor training progress

## Troubleshooting:

- **Import errors**: Make sure all files are uploaded/cloned correctly
- **CUDA errors**: Ensure GPU runtime is enabled
- **Memory errors**: Reduce batch size or use gradient accumulation
- **Data errors**: Check that WAV files are mono, 44.1kHz, and properly matched

## Next Steps:

1. Download trained models from the Results folder
2. Test inference with `proc_audio.py` on new audio
3. Experiment with different configurations (RNN1.json, RNN2.json)
4. Try different architectures or hyperparameters